<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/adding_C3/C3/W2/ungraded_labs/C3_W2_Lab_1_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

In [7]:
# !pip install tensorflow==2.5.0
# !pip install tensorflow-datasets

In [4]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [10]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info= True, as_supervised=True)

Dl Completed...: 100%|██████████| 1/1 [00:57<00:00, 57.07s/ url]
2021-09-03 12:42:59.110470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-03 12:42:59.110523: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-03 12:42:59.110655: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (prashant-thinkpad): /proc/driver/nvidia/version does not exist
2021-09-03 12:42:59.121063: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dataset imdb_reviews downloaded and prepared to /home/prashant/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [16]:
imdb.keys()

dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [12]:
train_data, test_data = imdb['train'], imdb['test']

In [51]:
training_sentences = []
training_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf-8'))
    training_labels.append(l.numpy())

testing_sentences = []
testing_labels = []

for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf-8'))
    testing_labels.append(l.numpy())

training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [53]:
# Let's check the shapes of our data
print(np.array(training_sentences).shape, training_labels.shape)
print(np.array(testing_sentences).shape, testing_labels.shape)

(25000,) (25000,)
(25000,) (25000,)


In [54]:
# defining constants
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen= max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, truncating=trunc_type)

In [56]:
# reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
reverse_word_index = {v:k for k, v in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print()
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received

This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [57]:
# defining the model
model = tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length= max_length),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(6, activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')
])

In [58]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [59]:
num_epochs = 10
model.fit(padded, training_labels, epochs = num_epochs, validation_data=(testing_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 9s 11ms/step - loss: 0.4949 - accuracy: 0.7403 - val_loss: 0.3763 - val_accuracy: 0.8322
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2403 - accuracy: 0.9053 - val_loss: 0.4096 - val_accuracy: 0.8229
Epoch 3/10
782/782 [==============================] - 9s 11ms/step - loss: 0.0945 - accuracy: 0.9762 - val_loss: 0.5133 - val_accuracy: 0.8046
Epoch 4/10
782/782 [==============================] - 9s 11ms/step - loss: 0.0239 - accuracy: 0.9969 - val_loss: 0.6074 - val_accuracy: 0.8061
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0066 - accuracy: 0.9994 - val_loss: 0.6939 - val_accuracy: 0.8032
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.7483 - val_accuracy: 0.8059
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 9.0785e-04 - accuracy: 1.0000 - val_loss: 0.7996 - val_accuracy: 0.80

In [61]:
# getting the embeddings weights
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # the 10k refers to our vocabulary and 16 is the embedding dimensions

(10000, 16)


In [76]:
import io

out_v = io.open('assets/vecs.tsv', 'w', encoding = 'utf-8')
out_m = io.open('assets/meta.tsv', 'w', encoding = 'utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [75]:
sentence = "Tensorflow is an amazing library. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[1, 7, 33, 478, 3315, 1200]]
